In [2]:
import chromadb
import string
import ast
import os

from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer
from transformers import T5Tokenizer, T5ForConditionalGeneration

import warnings
warnings.filterwarnings("ignore")

### Funciones necesarias

In [3]:
# Inicializar el stemmer en español
stemmer = SnowballStemmer("spanish")

# Cargar stopwords en español
stop_words = set(stopwords.words('spanish'))

# Definir caracteres de puntuación adicionales en español
puntuaciones = string.punctuation + "¿¡"

# Función de preprocesamiento mejorada
def preprocesar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()

    # Eliminar puntuaciones y números
    texto = "".join([char for char in texto if char not in puntuaciones and not char.isdigit()])

    # Tokenizar
    tokens = word_tokenize(texto)

    # Eliminar stopwords y aplicar stemming
    tokens_procesados = [stemmer.stem(t) for t in tokens if t not in stop_words]

    return " ".join(tokens_procesados)

In [4]:
# Función para buscar documentos en ChromaDB
def buscar_documentos(embedding_query, top_k=3):
    resultados = collection.query(query_embeddings=[embedding_query], n_results=top_k)
    return resultados  # Retornamos todo el resultado, incluyendo los metadatos

In [5]:
# Función para generar respuesta con T5
def generar_respuesta_t5(query, documentos, metadatos):
    # Crear el contexto con los documentos y metadatos
    contexto = " ".join(documentos)  # Unimos los documentos en un solo texto
    metadata_texto = f"Metadatos: {metadatos}"  # Incluimos los metadatos relevantes
    entrada = f"Pregunta: {query} Contexto: {contexto} {metadata_texto}"

    inputs = tokenizer_t5(entrada, return_tensors="pt", max_length=512, truncation=True)
    output = modelo_t5.generate(**inputs, max_length=150)

    return tokenizer_t5.decode(output[0], skip_special_tokens=True)

### Implementación de modelo de lenguaje T5

In [6]:
# Conectar a la base de datos vectorial
chroma_client = chromadb.PersistentClient(path="../db/")
collection = chroma_client.get_or_create_collection("entrevistas")

# Cargar el modelo T5
modelo_t5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")
tokenizer_t5 = T5Tokenizer.from_pretrained("google/flan-t5-large")

# Modelo con el que vamos a generar embeddings
modelo = SentenceTransformer("all-MiniLM-L6-v2")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
# Ejemplo de consulta
consulta_usuario = "¿Qué candidato presidencial habla mucho más sobre reducir la delincuencia?"
preproc_query_t5 = preprocesar_texto(consulta_usuario)
t5_query = modelo.encode(preproc_query_t5)

# Buscar documentos y metadatos en ChromaDB
resultados = buscar_documentos(t5_query)

# Obtener los documentos y metadatos
documentos_recuperados = resultados['documents'][0]  # Documentos relevantes
metadatos = resultados['metadatas'][0]  # Metadatos asociados

In [8]:
# Generar respuesta con T5
respuesta_t5 = generar_respuesta_t5(consulta_usuario, documentos_recuperados, metadatos)

print("Respuesta generada:", respuesta_t5)

Respuesta generada: Pedro granja
